In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
from queries import start_elastic_search, index_documents
from trec_files import make_trec_run
from trec_files import read_eval_files
from evaluation import trec_eval
from evaluation import mean_average_precision
from evaluation import print_trec_eval
import elasticsearch
import elasticsearch.helpers
import json
from elasticsearch import Elasticsearch
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from elasticsearch_dsl import Q
from elasticsearch_dsl import Search

print("done")

done


In [3]:
# Connect to the ElasticSearch server
es = elasticsearch.Elasticsearch(host='localhost')  # in case you use Docker, the host is 'elasticsearch'

# Index the collection into the index called 'genomics'
body = {} # no indexing options (leave default)
index_documents(es, 'data01/FIR-s05-medline.json', 'genomics-base', body)

/home/ut-student/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


(263080, [])

In [66]:
# connect to ES server             
es = elasticsearch.Elasticsearch('localhost')

# Write the results of the queries contained in the topic file `'data/training-queries-simple.txt'` 
# to the run file `'baseline.run'`, and name this test as `test01`
# make_trec_run(es, 'data01/FIR-s05-training-queries-simple.txt', 'baseline.run', "genomics-base", run_name='no_pseudo_01')
make_trec_run(es, 'data01/FIR-s05-training-queries-simple.txt', 'baseline.run', "genomics-base", run_name='pseudo_01', pseudo_relevance=True)

print("done")

"cyclin-dependent kinase inhibitor 1A (p21, Cip1)" in Homo sapiens 

"cyclin-dependent kinase inhibitor 1A (p21, Cip1)" in Homo sapiens tumor deficiency differential effects mice role age certain complex experimental 


ets variant gene 6 (TEL oncogene) in Homo sapiens 

ets variant gene 6 (TEL oncogene) in Homo sapiens phosphorylation map ser functional kinase regulation becomes bound constituent constitutive 


fibroblast growth factor 7 (keratinocyte growth factor) in Homo sapiens 

fibroblast growth factor 7 (keratinocyte growth factor) in Homo sapiens connective tissue activity factor growth wound around period suppression healing 


"glycine receptor, alpha 1 (startle disease/hyperekplexia, stiff man syndrome)" in Homo sapiens 

"glycine receptor, alpha 1 (startle disease/hyperekplexia, stiff man syndrome)" in Homo sapiens minor form major gene genetic defect generalized hereditary stiffness startle 


"major histocompatibility complex, class II, DQ beta 1" in Homo sapiens 

"maj

In [67]:
print_trec_eval('data01/FIR-s05-training-qrels.txt', 'baseline.run')

Results for baseline.run
mean success_at_1              0.1316
mean success_at_5              0.2105
mean success_at_10             0.3158
mean r_precision               0.08062
mean precision_at_1            0.1316
mean precision_at_5            0.06842
mean precision_at_10           0.05
mean precision_at_50           0.01842
mean precision_at_100          0.01105
mean precision_at_recall_00    0.1984
mean precision_at_recall_01    0.1923
mean precision_at_recall_02    0.1467
mean precision_at_recall_03    0.1308
mean precision_at_recall_04    0.1239
mean precision_at_recall_05    0.1183
mean precision_at_recall_06    0.1054
mean precision_at_recall_07    0.1036
mean precision_at_recall_08    0.08429
mean precision_at_recall_09    0.08381
mean precision_at_recall_10    0.08381
mean average_precision         0.1191


# Pseudo-Feedback Relevance

In [ ]:
initial_results = es.search(index='your_index_name', body=initial_query, size=10)

# Extract the initial retrieved documents' content
docs_content = [hit['_source']['field_to_extract'] for hit in initial_results['hits']['hits']]

# Combine the initial query with terms from the top retrieved documents
feedback_terms = []
for doc_content in docs_content:
    doc_tokens = word_tokenize(doc_content)
    doc_tokens = [porter.stem(token) for token in doc_tokens if token.isalpha() and token not in stop_words]
    feedback_terms.extend(doc_tokens)

# Calculate term frequencies for the feedback terms
feedback_term_freq = {}
for term in feedback_terms:
    if term in feedback_term_freq:
        feedback_term_freq[term] += 1
    else:
        feedback_term_freq[term] = 1

# Sort the feedback terms by their frequency
sorted_feedback_terms = sorted(feedback_term_freq.items(), key=lambda x: x[1], reverse=True)

# Choose the top feedback terms to expand the query
num_feedback_terms = 5
expanded_query = {
    "query": {
        "match": {
            "field_to_search": {
                "query": preprocess_text("your initial query"),
                "operator": "or"
            }
        }
    }
}

for term, _ in sorted_feedback_terms[:num_feedback_terms]:
    expanded_query["query"]["match"]["field_to_search"]["query"] += f" {term}"

# Perform a new search with the expanded query
new_results = es.search(index='your_index_name', body=expanded_query, size=10)

# You can now work with the new search results in 'new_results'


## Evaluate

In [ ]:
qrels_file = "data01/trec.qrels"
run_file = "data01/baseline.run"

# Read the relevant documents and retrieved documents
all_relevant, all_retrieved = read_eval_files(qrels_file, run_file)

# Evaluate the performance
map_score = mean_average_precision(all_relevant, all_retrieved)
print(f"Mean Average Precision (MAP): {map_score}")
